In [18]:
from sage.all import *
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.special import comb

from sage.combinat.combinatorial_map import combinatorial_map

from sage.structure.global_options import GlobalOptions
from sage.structure.parent import Parent
from sage.structure.unique_representation import UniqueRepresentation
from sage.categories.finite_enumerated_sets import FiniteEnumeratedSets
from sage.categories.infinite_enumerated_sets import InfiniteEnumeratedSets
from sage.categories.posets import Posets

from sage.rings.integer import Integer
from sage.rings.rational_field import QQ
from sage.combinat.permutation import Permutation, Permutations
from sage.combinat.words.word import Word
from sage.combinat.set_partition import SetPartitions
from sage.misc.latex import latex
from sage.misc.lazy_import import lazy_import
from typing import TYPE_CHECKING

from collections import Counter

def merge_sort(arr, start, end, compare):
    sorted_arr = [None] * (end - start + 1) 
    
    if start == end:
        sorted_arr[0] = arr[start]
        return sorted_arr

    left_start = start
    left_end = (start + end) // 2

    right_start = left_end + 1
    right_end = end

    left_sorted = merge_sort(arr, left_start, left_end, compare)
    right_sorted = merge_sort(arr, right_start, right_end, compare)

    i = 0  # current position in left part
    j = 0  # current position in right part

    k = 0

    while i < len(left_sorted) and j < len(right_sorted):
        a = left_sorted[i]
        b = right_sorted[j]
        decision = compare(a, b)
        if decision < 0:
            sorted_arr[k] = a
            i += 1
        else:
            sorted_arr[k] = b
            j += 1
        k += 1

    while i < len(left_sorted):
        sorted_arr[k] = left_sorted[i]
        i += 1
        k += 1

    while j < len(right_sorted):
        sorted_arr[k] = right_sorted[j]
        j += 1
        k += 1

    return sorted_arr

def dominanceCompare(x, y):
    poset = DyckWords().height_poset()
    if poset.le(x,y) and not poset.le(y,x):
        return -1
    if poset.le(y,x) and not poset.le(x,y):
        return 1
    else: 
        xp = x.peaks()
        yp = y.peaks()
        return (xp > yp) - (xp < yp)

def pVals(a, b, n):
    x = catalan_number(a);
    return x;

class Meander:
    def __init__(self, x, y):
        self.topDyck = x;
        self.botDyck = y;
        self.size = len(x)
        top_arches = [tuple(a0 + 1 if i == 0 else a0 for i, a0 in enumerate(a))
                       for a in x.tunnels()]
        bot_arches = [tuple(b0 + 1 if i == 0 else b0 for i, b0 in enumerate(b))
                       for b in y.tunnels()]

        self.topPerm = Permutation(top_arches)
        self.botPerm = Permutation(bot_arches)
        
        self.perm = self.botPerm.left_action_product(self.topPerm)
        self.order = len(self.perm.cycle_type())/2
        
    def getPerm(self):
        return self.perm.cycle_tuples();

    def getTop(self):
        return self.topPerm.cycle_tuples();

    def getBot(self):
        return self.botPerm.cycle_tuples();

    def getOrder(self):
        return self.order;

    def getCycleType(self):
        return self.perm.cycle_type()


def dyck_words(n):
    out=[]
    def rec(w,o,c):
        if o==n and c==n:
            out.append(DyckWord(w)); return
        if o<n: rec(w+'(', o+1, c)
        if c<o: rec(w+')', o, c+1)
    rec('',0,0); return out

def meanderMatrix(n):
    R = PolynomialRing(QQ, 't')
    t = R.gen();
    q = (t)
    dws = dyck_words(n)
    arches = merge_sort(dws, 0, len(dws)-1, dominanceCompare)
    l = len(arches)
    mM = [[0]*l for _ in range(l)]
    
    for i in range(l):
        for j in range(i, l): 
            value = Meander(arches[i], arches[j]).getOrder()
            mM[i][j] = q^value
            mM[j][i] = q^value
    return mM

def stanleyPoly(m):
    # R = PolynomialRing(QQ, m.size, "p")
    p = SymmetricFunctions(QQ).p()
    # p = R.gens()
    entry = 1
    counts = Counter(m.getCycleType())
    for k in counts:
        entry *= p[k]**(Integer(counts[k]/2))
    return entry

def stanleyMatrix(n):
    dws = dyck_words(n)
    arches = merge_sort(dws, 0, len(dws)-1, dominanceCompare)
    l = len(arches)
    hM = [[0]*l for _ in range(l)]
    
    for i in range(l):
        for j in range(i, l): 
            m = Meander(arches[i], arches[j])
            value = stanleyPoly(m)
            hM[i][j] = value
            hM[j][i] = value
    return hM

def DFChebyshev(gen, k):
    return chebyshev_U(k, gen/2)

def c_nh(n, h):
    return comb(n,(n-h)/2, exact=true) - comb(n, (n-h)/2 - 1, exact=true)

def meanderDeterminant(M, n):
    R = PolynomialRing(QQ, 't')
    t = R.gen();
    mu = (t)
    ret = DFChebyshev(mu, 0)
    for i in range(n):
        m = i+1
        ret *= DFChebyshev(mu, m)^(c_nh(2*n,2*m)-c_nh(2*n,2*m+2))
    return ret

def terms(M, col = 1, row = 1):
    return [x[:col-1] + x[col:] for x in M[0:row-1] + M[row:]]

def detNxN(M):
    N = len(M[0])
    M = [row[:] for row in M]
    
    if (N == 2): 
        M = np.array(M)
        return M[0][0] * M[1,1] - M[0][1] * M[1][0]
    
    else: 
        rowValues = M[:1][0]
        colsSigns = [1 if (col % 2 == 0) else -1 for col in range(N)]
        colsDets = [detNxN(terms(M, col + 1)) for col in range(N)]
        return sum([rowValues[col] * colsSigns[col] * colsDets[col] for col in range(N)])

def bareissDet(M):
    n = len(M)
    A = [row[:] for row in M]  # make a copy, because A will be modified

    for k in range(n - 1):
        pivot = A[k][k]
        denom = 1 if k == 0 else A[k - 1][k - 1]

        for i in range(k + 1, n):
            for j in range(k + 1, n):
                A[i][j] = (A[i][j] * pivot - A[i][k] * A[k][j]) / denom
            A[i][k] = 0

    return numerator(A[-1][-1])

def schurPositive(f):
    s = Sym.schur()
    f = s(f)
    coeff = f.coefficient()
    for c in coeff:
        if c < 0: 
            return false
    return true;

def trace_M_squared(M):
    n = len(M)
    MM = [[sum(M[i][k] * M[k][j] for k in range(n)) for j in range(n)] for i in range(n)]
    return sum(MM[i][i] for i in range(n))
    

#m = Meander(y, x.reverse());
#print(m.topPerm);
#print(type(m.topPerm[0]));
#print(m.getTop());
#print(m.getBot());
#print(m.getPerm());
#d = dyck_words(6)
#mtest = Meander(DyckWord(d[0]), DyckWord(d[1]))
# print(mtest.getPerm());
# dws = DyckWords(3)
# mn = meanderMatrix(7)
# plt.matshow(mn)
# plt.show()

Sym = SymmetricFunctions(QQ)
s = Sym.schur()
R = PolynomialRing(QQ, 't')
t = R.gen();
mu = (t)

x=DyckWord("(())()")
y=DyckWord("()()()")
m=Meander(x,y)

p = PolynomialRing(QQ, m.size, "p")
test = p["a"]^2

M = meanderMatrix(4)

# print("regular det: ", detNxN(M))
dM = meanderDeterminant(M, 4)
# print("dF det: ", dM)
print("The factorization of meander determinant: ", dM.factor())
# print("bareiss det: ", bareissDet(M))
# print("regular det: ", detNxN(M))
H = stanleyMatrix(4)
dH = bareissDet(H)
f = dH.factor()
# print("bareiss det of H: ", dH)
print("factorization of det(H): ", f) 
# print(list(f))
# print(dH.degree())
# print(dH.coefficients())
# print(dH.is_schur_positive())

# schur_expansion = dH.expand(backend="sage").express(s)
# plt.matshow(M)

sumTerms = 0
for i in range(len(H)):
    for j in range(len(H)):
        sumTerms += H[i][j]
print("is sum schur positive: ", sumTerms.is_schur_positive())
print("trace of M squared: ", trace_M_squared(M))
trHH = trace_M_squared(H)
print("trace of H squared: ", trHH)
print("is trace of H^2 schur positive: ", trHH.is_schur_positive())





The factorization of meander determinant:  (t - 1)^13 * (t + 1)^13 * t^14 * (t^2 - t - 1) * (t^2 + t - 1) * (t^2 - 2)^6
factorization of det(H):  (-1) * (-p[1, 1, 1, 1, 1, 1, 1, 1] + 2*p[2, 2, 1, 1, 1, 1] + p[2, 2, 2, 2] + 2*p[3, 1, 1, 1, 1, 1] - 2*p[3, 2, 2, 1] - 4*p[4, 2, 1, 1] + 2*p[4, 4])^2 * (-p[1, 1, 1, 1, 1, 1, 1, 1] + 4*p[2, 2, 1, 1, 1, 1] - p[2, 2, 2, 2] - 4*p[4, 2, 1, 1] + 2*p[4, 4])^2 * (-p[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] + 2*p[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] + 6*p[2, 2, 1, 1, 1, 1, 1, 1, 1, 1] + 2*p[2, 2, 2, 1, 1, 1, 1, 1, 1] + 7*p[2, 2, 2, 2, 1, 1, 1, 1] - 8*p[2, 2, 2, 2, 2, 1, 1] + 2*p[2, 2, 2, 2, 2, 2] - 2*p[3, 1, 1, 1, 1, 1, 1, 1, 1, 1] - 16*p[3, 2, 1, 1, 1, 1, 1, 1, 1] - 22*p[3, 2, 2, 1, 1, 1, 1, 1] + 8*p[3, 2, 2, 2, 1, 1, 1] + 4*p[3, 2, 2, 2, 2, 1] + 12*p[3, 3, 1, 1, 1, 1, 1, 1] + 16*p[3, 3, 2, 1, 1, 1, 1] - 12*p[3, 3, 2, 2, 1, 1] + 5*p[4, 1, 1, 1, 1, 1, 1, 1, 1] - 2*p[4, 2, 1, 1, 1, 1, 1, 1] - 2*p[4, 2, 2, 2, 1, 1] - 3*p[4, 2, 2, 2, 2] - 2*p[4, 3, 1, 1, 1, 1, 1]